# Problem Set 5: Derivatives

# Task 1: From Forwards to Interest

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from datetime import datetime
from scipy.interpolate import make_interp_spline, BSpline
import matplotlib.dates as mdates
import statsmodels.api as sm
from sympy.solvers import solve
from sympy import Symbol
warnings.filterwarnings('ignore')

In [5]:
data = pd.read_csv(r'Data/ToF_2021_PS5-data-1.csv', delimiter = ';', index_col = 'date')
print(data.head(n = 10))

            UBS_CHINA_EQUITY_FUND
date                             
10.08.2018                  17862
13.08.2018                  17784
14.08.2018                  17552
15.08.2018                  17424
16.08.2018                  17015
17.08.2018                  16797
20.08.2018                  16640
21.08.2018                  16840
22.08.2018                  17088
23.08.2018                  17247


In [17]:
# Exogenous Variables
n = 3
S_0 = 1000000
rf_rate = -0.0076
performance_fee = 0.1
binomial_step = 1
months_per_year = 12
days_per_year = 252

### a)

In [16]:
returns = data.pct_change()
returns['Log Return'] = np.nan
returns.rename(columns = {'UBS_CHINA_EQUITY_FUND': 'Return'}, inplace = True)
for i in range(0, len(returns.index)):
    returns.iloc[i, 1] = np.log(1 + returns.iloc[i, 0])
print(returns.head(n=10))

              Return  Log Return
date                            
10.08.2018       NaN         NaN
13.08.2018 -0.004367   -0.004376
14.08.2018 -0.013045   -0.013131
15.08.2018 -0.007293   -0.007319
16.08.2018 -0.023473   -0.023753
17.08.2018 -0.012812   -0.012895
20.08.2018 -0.009347   -0.009391
21.08.2018  0.012019    0.011948
22.08.2018  0.014727    0.014619
23.08.2018  0.009305    0.009262


In [25]:
annualized_mean = days_per_year * returns['Log Return'].mean()
annualized_std = days_per_year**0.5 * returns['Log Return'].std()
u = np.exp(annualized_std)
d = np.exp(-annualized_std)
print(f'Annualized Mean: {round(annualized_mean * 100, 2)}%')
print(f'Annualized Standard Deviation: {round(annualized_std * 100, 2)}%')
print(f'Up-Factor: {round(u * 100, 2)}%')
print(f'Down-Factor: {round(d * 100, 2)}%')

Annualized Mean: 4.21%
Annualized Standard Deviation: 21.93%
Up-Factor: 124.52%
Down-Factor: 80.31%


### b)

In [45]:
q = (annualized_mean*(1/days_per_year) - np.log(d)) / (np.log(u) - np.log(d))
print(f'Upward Probability: {round(q*100, 4)}%')
print(f'Downward Probability: {100-round(q*100, 4)}%')

Upward Probability: 50.0381%
Downward Probability: 49.9619%


In [39]:
# Period 0
S_0 = 1000000
# Period 1
S_1_up = S_0 * u
S_1_down = S_0 * d
# Period 2
S_2_up_up = S_1_up * u
S_2_up_down = S_1_up * d
S_2_down_up = S_1_down * u
S_2_down_down = S_1_down * d
# Period 3
S_3_up_up_up = S_2_up_up * u
S_3_up_up_down = S_2_up_up * d
S_3_up_down_up = S_2_up_down * u
S_3_up_down_down = S_2_up_down * d
S_3_down_up_up = S_2_down_up * u
S_3_down_up_down = S_2_down_up * d
S_3_down_down_up = S_2_down_down * u
S_3_down_down_down = S_2_down_down * d

In [41]:
# Period 0
print(f'S0: {round(S_0, 2)}')
# Period 1
print(f'S1 (U): {round(S_1_up, 2)}')
print(f'S1 (D): {round(S_1_down, 2)}')
# Period 2
print(f'S2 (UU): {round(S_2_up_up, 2)}')
print(f'S2 (UD): {round(S_2_up_down, 2)}')
print(f'S2 (DU): {round(S_2_down_up, 2)}')
print(f'S2 (DD): {round(S_2_down_down, 2)}')
# Period 3
print(f'S3 (UUU): {round(S_3_up_up_up, 2)}')
print(f'S3 (UUD): {round(S_3_up_up_down, 2)}')
print(f'S3 (UDU): {round(S_3_up_down_up, 2)}')
print(f'S3 (UDD): {round(S_3_up_down_down, 2)}')
print(f'S3 (DUU): {round(S_3_down_up_up, 2)}')
print(f'S3 (DUD): {round(S_3_down_up_down, 2)}')
print(f'S3 (DDU): {round(S_3_down_down_up, 2)}')
print(f'S3 (DDD): {round(S_3_down_down_down, 2)}')

S0: 1000000
S1 (U): 1245219.52
S1 (D): 803071.25
S2 (UU): 1550571.65
S2 (UD): 1000000.0
S2 (DU): 1000000.0
S2 (DD): 644923.44
S3 (UUU): 1930802.08
S3 (UUD): 1245219.52
S3 (UDU): 1245219.52
S3 (UDD): 803071.25
S3 (DUU): 1245219.52
S3 (DUD): 803071.25
S3 (DDU): 803071.25
S3 (DDD): 517919.47


### c)

In [53]:
# Period 1
NPV_1 = 0
if S_1_up > S_0:
    NPV_1 += 1/(1-rf_rate) * q * (S_1_up-S_0) * 0.1
if S_1_down > S_0:
    NPV_1 += 1/(1-rf_rate) * (1-q) * (S_1_down-S_0) * 0.1
print(f'Expected NPV of CFs in Period 1: {round(NPV_1, 2)}')

Expected NPV of CFs in Period 1: 12177.77


In [54]:
# Period 2
NPV_2 = 0
if S_2_up_up > S_1_up:
    NPV_2 += 1/(1-rf_rate)**2 * q**2 * (S_2_up_up-S_1_up) * 0.1
if S_2_up_down > S_1_up:
    NPV_2 += 1/(1-rf_rate)**2 * q * (1-q) * (S_2_up_down-S_1_up) * 0.1
if S_2_down_up > S_0:
    NPV_2 += 1/(1-rf_rate)**2 * q * (1-q) * (S_2_down_up-S_1_down) * 0.1
if S_2_down_down > S_0:
    NPV_2 += 1/(1-rf_rate)**2 * (1-q)**2 * (S_2_down_down-S_1_down) * 0.1
print(f'Expected NPV of CFs in Period 2: {round(NPV_2, 2)}')

Expected NPV of CFs in Period 2: 7530.55


In [55]:
# Period 2
NPV_3 = 0
if S_3_up_up_up > S_2_up_up:
    NPV_3 += 1/(1-rf_rate)**3 * q**3 * (S_3_up_up_up-S_2_up_up) * 0.1
if S_3_up_up_down > S_2_up_up:
    NPV_3 += 1/(1-rf_rate)**3 * q**2 * (1-q) * (S_3_up_up_down-S_2_up_up) * 0.1
if S_3_up_down_up > S_1_up:
    NPV_3 += 1/(1-rf_rate)**3 * q * (1-q) * q * (S_3_up_down_up-S_2_up_down) * 0.1
if S_3_up_down_down > S_1_up:
    NPV_3 += 1/(1-rf_rate)**3 * q * (1-q) * (1-q) * (S_3_up_down_down-S_2_up_down) * 0.1
if S_3_down_up_up > S_2_down_up:
    NPV_3 += 1/(1-rf_rate)**3 * (1-q) * q**2 * (S_3_down_up_up-S_2_down_up) * 0.1
if S_3_down_up_down > S_2_down_up:
    NPV_3 += 1/(1-rf_rate)**3 * (1-q) * q * (1-q) * (S_3_down_up_down-S_2_down_up) * 0.1
if S_3_down_down_up > S_0:
    NPV_3 += 1/(1-rf_rate)**3 * (1-q)**2 * q * (S_3_down_down_up-S_2_down_down) * 0.1
if S_3_down_down_down > S_0:
    NPV_3 += 1/(1-rf_rate)**3 * (1-q)**3 * (S_3_down_down_down-S_2_down_down) * 0.1
print(f'Expected NPV of CFs in Period 3: {round(NPV_3, 2)}')

Expected NPV of CFs in Period 3: 7655.46


### d)

In [56]:
k = 0.1
utility_1 = NPV_1**(1-k) / (1-k)
utility_2 = NPV_2**(1-k) / (1-k)
utility_3 = NPV_3**(1-k) / (1-k)
print(f'Expected Utility from CFs in Period 1: {round(utility_1, 2)}')
print(f'Expected Utility from CFs in Period 2: {round(utility_2, 2)}')
print(f'Expected Utility from CFs in Period 3: {round(utility_3, 2)}')

Expected Utility from CFs in Period 1: 5281.64
Expected Utility from CFs in Period 2: 3426.9
Expected Utility from CFs in Period 3: 3478.02


### e)

In [66]:
c = Symbol('c')
coupon = solve((1/(1+rf_rate) * c + 1/(1+rf_rate)**2 * c + 1/(1+rf_rate)**3 * c)**(1-k) / (1-k) - utility_1 - utility_2 - utility_3, c)[0]
print(f'Utility-Indifferent Fixed Annual Coupon: {round(coupon, 2)}')

Utility-Indifferent Fixed Annual Coupon: 10122.08


### f)

see solution sheet